# API Face Samples

## Objective
The script uses the Azure Face API to detect faces in an image, create and populate a face list, and create and train a person group.

In [ ]:
import os, requests
import uuid
import time
from PIL import Image
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel

def detect_faces(subscription_key, endpoint, image_path):
    with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"x-ms-useragent": "create-face-collection/1.0.0"}) as face_client:
        with open(image_path, 'rb') as image_data:
            detected_faces = face_client.detect(
                    image_content=image_data.read(),
                    detection_model=FaceDetectionModel.DETECTION_03,
                    recognition_model=FaceRecognitionModel.RECOGNITION_04,
                    return_face_id=True,
                )
        return detected_faces

# Function to enlarge the bounding box
def enlarge_bounding_box(face_rectangle, image_width, image_height, enlargement_factor=1.2):
    left = max(0, face_rectangle['left'] - (face_rectangle['width'] * (enlargement_factor - 1) / 2))
    top = max(0, face_rectangle['top'] - (face_rectangle['height'] * (enlargement_factor - 1) / 2))
    width = min(image_width - left, face_rectangle['width'] * enlargement_factor)
    height = min(image_height - top, face_rectangle['height'] * enlargement_factor)
    return {'left': int(left), 'top': int(top), 'width': int(width), 'height': int(height)}

# Function to create a face list and add faces to it
def create_face_list(subscription_key, endpoint, face_list_id, faces, image_path, image_width, image_height):
    face_list_url = endpoint + f'/face/v1.0/facelists/{face_list_id}'
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Content-Type': 'application/json',
        'x-ms-useragent': 'create-face-collection/1.0.0'
    }
    body = {
        'name': 'my_face_list',
        'userData': 'A list of faces from an image',
        'recognitionModel': 'recognition_04'
    }
    response = requests.put(face_list_url, headers=headers, json=body)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"Error in create_face_list: {err.response.text}")
        raise
    
    # Add faces to the face list
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Content-Type': 'application/octet-stream',
        'x-ms-useragent': 'create-face-collection/1.0.0'
    }
    add_face_url = face_list_url + '/persistedFaces'
    face_count = 0
    for face in faces:
        face_rectangle = enlarge_bounding_box(face['faceRectangle'], image_width, image_height)
        params = {
            'targetFace': f"{face_rectangle['left']},{face_rectangle['top']},{face_rectangle['width']},{face_rectangle['height']}",
            'detectionModel': 'detection_03'
        }
        with open(image_path, 'rb') as image_data:
            response = requests.post(add_face_url, params=params, headers=headers, data=image_data)
        try:
            response.raise_for_status()
            face_count += 1
        except requests.exceptions.HTTPError as err:
            print(f"Error in add_face_to_list: {err.response.text}")
            raise
    print(f"Total faces added to face list: {face_count}")

# Function to create a person group, add persons to it, and train the group
def create_person_group(subscription_key, endpoint, person_group_id, faces, image_path, image_width, image_height):
    person_group_url = endpoint + f'/face/v1.0/persongroups/{person_group_id}'
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Content-Type': 'application/json',
        'x-ms-useragent': 'create-face-collection/1.0.0'
    }
    body = {
        'name': 'my_person_group',
        'userData': 'A group of persons from an image',
        'recognitionModel': 'recognition_04'
    }
    response = requests.put(person_group_url, headers=headers, json=body)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"Error in create_person_group: {err.response.text}")
        raise
    
    # Add persons to the person group
    add_person_url = person_group_url + '/persons'
    person_count = 0
    for face in faces:
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Content-Type': 'application/json',
            'x-ms-useragent': 'create-face-collection/1.0.0'
        }
        body = {
            'name': 'person_' + str(face['faceId']),
            'userData': 'A person detected in the image'
        }
        response = requests.post(add_person_url, headers=headers, json=body)
        try:
            response.raise_for_status()
            person_count += 1
        except requests.exceptions.HTTPError as err:
            print(f"Error in add_person_to_group: {err.response.text}")
            raise
        person_id = response.json()['personId']
        
        # Add face to person
        add_face_url = person_group_url + f'/persons/{person_id}/persistedFaces'
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Content-Type': 'application/octet-stream',
            'x-ms-useragent': 'create-face-collection/1.0.0'
        }
        face_rectangle = enlarge_bounding_box(face['faceRectangle'], image_width, image_height)
        params = {
            'targetFace': f"{face_rectangle['left']},{face_rectangle['top']},{face_rectangle['width']},{face_rectangle['height']}",
            'detectionModel': 'detection_03'
        }
        with open(image_path, 'rb') as image_data:
            response = requests.post(add_face_url, params=params, headers=headers, data=image_data)
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f"Error in add_face_to_person: {err.response.text}")
            raise

    print(f"Total persons added to person group: {person_count}")

    # Train the person group
    train_url = endpoint + f'/face/v1.0/persongroups/{person_group_id}/train'
    response = requests.post(train_url, headers={'Ocp-Apim-Subscription-Key': subscription_key, 'x-ms-useragent': 'create-face-collection/1.0.0'})
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"Error in train_person_group: {err.response.text}")
        raise
    
    # Check training status
    status_url = endpoint + f'/face/v1.0/persongroups/{person_group_id}/training'
    while True:
        response = requests.get(status_url, headers={'Ocp-Apim-Subscription-Key': subscription_key, 'x-ms-useragent': 'create-face-collection/1.0.0'})
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f"Error in check_training_status: {err.response.text}")
            raise
        status = response.json()
        if status['status'] == 'succeeded':
            break
        elif status['status'] == 'failed':
            raise Exception('Training failed')
        time.sleep(1)

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'

# Detect faces in the image
faces = detect_faces(FACE_KEY, FACE_ENDPOINT, image_path)

# Create a unique face list ID and person group ID
face_list_id = str(uuid.uuid4())
person_group_id = str(uuid.uuid4())

image_width, image_height = get_image_dimensions(image_path)

# Create a face list and add faces to it
create_face_list(FACE_KEY, FACE_ENDPOINT, face_list_id, faces, image_path, image_width, image_height)

# Create a person group, add persons to it, and train the group
create_person_group(FACE_KEY, FACE_ENDPOINT, person_group_id, faces, image_path, image_width, image_height)

print(f"Face list ID: {face_list_id}")
print(f"Person group ID: {person_group_id}")